<a href="https://colab.research.google.com/github/lukaszlewickii/spark_labs/blob/main/df1_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataFrame

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#pyspark-sql-module

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=b0d5e6ab038d74fa965a8ca615eda0c44192de72ca7a9508b2db3dd4d1d4d118
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder \
    .appName('DataFrame_1') \
    .master('local[*]') \
    .getOrCreate()

In [ ]:
sc = spark.sparkContext

In [ ]:
# Wczytanie DataFrame bezposrednio z JSON oraz CSV
people = spark.read.json('people.json')
employees = spark.read.json('employees.json')
people_txt = spark.read.option("inferSchema", "true").csv('people.txt')

In [ ]:
people_txt.show()

+-------+----+
|    _c0| _c1|
+-------+----+
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



****

### DataFrame to lista wierszy

In [ ]:
newPerson1 = Row(name='Greg', age=32)

In [ ]:
newPerson1

Row(name='Greg', age=32)

In [ ]:
newPerson1.name

'Greg'

In [ ]:
newPerson1.age

32

In [ ]:
newPerson1['age']

32

In [ ]:
'age' in newPerson1

True

In [ ]:
newPerson = Row("age", "name")

In [ ]:
newPerson2 = newPerson(24, 'Alice')

In [ ]:
newPerson2

Row(age=24, name='Alice')

In [ ]:
newPerson3 = newPerson(None, None)
newPerson4 = newPerson(33, None)
newPerson5 = newPerson(None, 'Peter')
newPerson6 = newPerson(32, 'Peter')
newPerson7 = newPerson(40, 'Greg')

In [ ]:
newPeopleDF = spark.createDataFrame([newPerson1, newPerson2, newPerson3, newPerson4, newPerson5, newPerson6, newPerson7])

In [ ]:
newPeopleDF.show()

+----+----+
|name| age|
+----+----+
|Greg|  32|
|  24|NULL|
|NULL|NULL|
|  33|NULL|
|NULL|NULL|
|  32|NULL|
|  40|NULL|
+----+----+



### Przechodzenie RDD <-> DF

In [ ]:
type(people)

pyspark.sql.dataframe.DataFrame

In [ ]:
people

DataFrame[age: bigint, name: string]

In [ ]:
people.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

DF -> RDD

In [ ]:
type(people.rdd)

pyspark.rdd.RDD

In [ ]:
people.rdd.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

In [ ]:
people.rdd.map(tuple).collect()

[(None, 'Michael'), (30, 'Andy'), (19, 'Justin')]

RDD -> DF

In [ ]:
people.rdd.toDF()

DataFrame[age: bigint, name: string]

In [ ]:
people.rdd.map(tuple).toDF().collect()

[Row(_1=None, _2='Michael'), Row(_1=30, _2='Andy'), Row(_1=19, _2='Justin')]

Podanie schematu wprost przy tworzeniu DF

Typy danych: http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#module-pyspark.sql.types <br>
https://spark.apache.org/docs/latest/sql-programming-guide.html#data-types <br>
Kilka podstawowych: IntegerType, DoubleType, FloatType, StringType, BooleanType, NullType

In [ ]:
from pyspark.sql.types import IntegerType, StringType, StructType, StructField
schema = StructType([StructField("V1", IntegerType()), StructField("V2", StringType())])
df = spark.createDataFrame([[1,2],[3,4]], schema)

In [ ]:
# Data Frame nadal jest lista wierszy
df.show()

+---+---+
| V1| V2|
+---+---+
|  1|  2|
|  3|  4|
+---+---+



> **TODO**: Z podanego RDD utworz DataFrame. <br>
Wskazowka: DataFrame stworzymy latwo z RDD Rows<br>

In [ ]:
myRDD = sc.textFile('people.txt')
print(type(myRDD))

<class 'pyspark.rdd.RDD'>


In [ ]:
myRDD

people.txt MapPartitionsRDD[112] at textFile at NativeMethodAccessorImpl.java:0

#### Ogolne wiadomosci na temat danych
printSchema, show, columns, dtypes <br>
Znane z RDD np: count, take, head

In [ ]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [ ]:
people.printSchema() ## Print the schema in a tree format

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [ ]:
employees.printSchema()

root
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)



In [ ]:
people_txt.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: double (nullable = true)



In [ ]:
people_txt.show(1)

+-------+----+
|    _c0| _c1|
+-------+----+
|Michael|29.0|
+-------+----+
only showing top 1 row



In [ ]:
people.show(1)

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
+----+-------+
only showing top 1 row



In [ ]:
people_txt.show(1)

+-------+----+
|    _c0| _c1|
+-------+----+
|Michael|29.0|
+-------+----+
only showing top 1 row



Liczba wierszy

In [ ]:
people.count()

3

Lista kolumn

In [ ]:
people.columns

['age', 'name']

Lista kolumn wraz z typami danych

In [ ]:
people.dtypes

[('age', 'bigint'), ('name', 'string')]

#### Odwolania do poszczegolnych kolumn

In [ ]:
people.age

Column<'age'>

In [ ]:
people['age']

Column<'age'>

In [ ]:
people[0]

Column<'age'>

#### Dodanie/usuniecie kolumny
withColumn, drop

In [ ]:
people.show(2)

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
+----+-------+
only showing top 2 rows



In [ ]:
people.withColumn(colName = 'ageNextYear', col = people.age +1).show(2)

+----+-------+-----------+
| age|   name|ageNextYear|
+----+-------+-----------+
|NULL|Michael|       NULL|
|  30|   Andy|         31|
+----+-------+-----------+
only showing top 2 rows



In [ ]:
people.drop('age').show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



#### Zmiany nazwy kolumny

DataFrame people_txt ma domyslne nazwy kolumn ze Sparka. Niewiele nam one mowia.

In [ ]:
people_txt = people_txt.withColumnRenamed('_c0', 'name').withColumnRenamed('_c1', 'age')
people_txt.show(1)

#### Podstawowe statystyki kolumn w DataFrames
describe

In [ ]:
people.describe()

In [ ]:
employees.describe('salary').show()

#### Braki danych
isNull, isNotNull<br>
fillna, dropna, replace

In [ ]:
newPeopleDF.show()

In [ ]:
newPeopleDF.filter(newPeopleDF.age.isNull()).show()

In [ ]:
newPeopleDF.filter(newPeopleDF.age.isNotNull()).show()

In [ ]:
newPeopleDF.fillna(-1).show()

In [ ]:
newPeopleDF.fillna({'age':-1, 'name':'unknown'}).show()

In [ ]:
newPeopleDF.fillna({'name':'unknown'}).replace('unknown', 'NN').show()

In [ ]:
newPeopleDF.dropna().show()

In [ ]:
newPeopleDF.dropna(subset='age').show()

In [ ]:
newPeopleDF.dropna?

#### Funkcje wprost ze skladni SQL
select, where (wymienne z filter), orderBy

In [ ]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [ ]:
people.\
select('name').\
where(people.name.like('%n%')).\
orderBy(people.age.asc()).\
show()

+------+
|  name|
+------+
|Justin|
|  Andy|
+------+



> **TODO**: Wyswietl imiona ludzi ze zbioru 'people' starszych niz 29 lat

#### Operacje na zbiorach
union - dziala jak UNION ALL w SQL. <br>
intersect (INTERSECT z SQLa), subtract (EXCEPT z SQLa)

In [ ]:
people.union?

In [ ]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [ ]:
people_txt.show()

+-------+----+
|    _c0| _c1|
+-------+----+
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



In [ ]:
newPeopleDF.show()

+----+----+
|name| age|
+----+----+
|Greg|  32|
|  24|NULL|
|NULL|NULL|
|  33|NULL|
|NULL|NULL|
|  32|NULL|
|  40|NULL|
+----+----+



In [ ]:
from google.colab import drive
drive.mount('/content/drive')